#### Margaret Antonio 16.08.24

#### Overview

MAGenTA is a command-line based tool for Transposon Insertion Sequencing (Tn-Seq) data analysis. It was developed to analyze data generated by the method outlined in [van Opijnen 2009](http://www.nature.com/nmeth/journal/v6/n10/abs/nmeth.1377.html), which unlike other transposon insertion sequencing methods (cite here), assesses organismal fitness by profiling the mutant library at two time points, typically before and after a challenge (i.e. an antibiotic) is presented. The MAGenTA pipeline tools differ from other Tn-Seq analysis tools in that they perform comparative analyses that include this fitness measurement, rather than only looking at insertion representation.
</break>

Say something about Galaxy instance here.

</break>
INSERT WORKFLOW HERE FROM GALAXY



***
### <center>Summary of MAGenTA Tools</center>
***


| Tool           | Function                                                                                                                                                    | Research Questions Addressed                                                                                                                       |
|----------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------|
| DataOverview   | Produces a summary file of information about TA sites and insertions in the genome, libraries, and gene/intergenic regions                                  | What is the TA and insertion distribution in the genome for genes and non-gene regions?                                                            |
| SlidingWindow  | Scans the genome data with set window size and step to perform aggregate fitness and essentiality assessments at each window.                               | Which unannotated (intergenic or gene domain) regions are important or essential?                                                                  |
| CustomWindow   | Given a list of start and end coordinates, will return aggregate fitness and essentiality test results. Useful for recalculating info for merged regions    | What is the importance or essentiality of specific regions of interest?                                                                            |
| CompareGenes   | Creates a gene comparison file with significant stats for two Tn-Seq experiments of the same ref genome. Need aggregate output                              | How can we compare genes for a given strain in two different conditions? Which genes are conditionally important?                                  |
| CompareStrains | Creates a gene comparison file with significant stats for two Tn-Seq experiments of different strains (genomes). Need conversion file and aggregate output. | How can we compare homologous genes for two strains? Which genes have a strain-dependent importance?                                               |
| CompareWindows | Creates a comparison file with significance statistics for two Tn-Seq experiments of different strains (genomes). Need sliding window output                | How can we compare intergenic or gene domain regions for the same strain in two conditions?                                                        |
| Grouper        | Group consecutive sliding window regions that have the same importance/essentiality                                                                         | If consecutive unannotated regions have the same characteristics of importance and/or essentiality, how can we group them to form a single region? |
| GvizTnseq      | Tn-seq visualization functions based on an adpatation of the Gviz R package                                                                                 | How can we visualize multiple Tn-Seq data features for a given region of interest and highlight special regions?                                   |



***
### <center>DataOverview</center>
<center>Characterizing the reference genome and genome-wide insertion data</center>
***

Prior to Tn-seq data analysis it is important to understand the genome-wide scale and context of the data. Large differences in sequencing coverage, insertion representation, and GC content between genomes can affect the robustness of the comparative analyses performed by MAGenTA. Tn-Seq is also generally performed with replicate libraries, where differences (if they exist) would be important to know. The DataOverview tool produces a summary of genome-wide insertion representation and inherent genome characteristics. The input data are queried for TA sites and insertions with respect to

    1. The genome
    2. Open reading frames denoted by start and stop codons
    3. Annotated genes from the Genbank record
    4. Intergenic regions

#### USAGE

```perl
perl dataOverview.pl -i path/to/results/files -f refGenome.fa -r refGenome.gbk
```

    - i     a directory of files, each containing the insertion sites, read counts, fitness values, and other information about single insertion mutants for that library
    - r     the Genbank record for the organism
    - f     the fasta sequence file for the organism. 


#### EXAMPLE

Although Streptococcus pneumoniae TIGR4 and Taiwan-19F have highly similar inherent genome characteristics (size, number of genes, GC content, etc.), they differ for genome-wide insertion representation. The DataOverview tool also outputs a summary of genome and gene-wide insertion representation for each library, including how many insertions would be filtered out given a cutoff requirement for the number of reads representing each insertion. 

|                                          | TIGR4       | Taiwan-19F |
|------------------------------------------|-------------|------------|
| Genbank Record                           | NC_003028b2 | NC_0012469 |
| Genome Size (bp)                         | 2160842     | 2112148    |
| GC content                               | 39.70%      | 39.77%     |
| Number of Genes                          | 2390        | 2204       |
| Total number of TA sites                 | 141459      | 137693     |
| Genome coverage by TA sites              | 6.55%       | 6.52%      |
| Largest gap between TA sites (bp)        | 252         | 206        |
| Genome coverage by insertions            | 1.33%       | 3.68%      |
| TA site coverage by insertions           | 32.80%      | 73.12%     |
| TA site coverage by filtered insertions* | 20.28%      | 56.45%     |
| Largest gap between insertions (bp)      | 14996       | 6708       |
| Smallest ORF (bp)                        | 6           | 6          |
| Largest ORF (bp)                         | 220         | 220        |
| Max number TA sites in an ORF            | 10          | 10         |
| Number of ORFs without TA sites          | 6997        | 6701       |
| Average gene length                      | 803         | 834        |
| Minimum gene length                      | 30          | 65         |
| Max gene length                          | 14330       | 6701       |
| Average insertions in a gene             | 9.84        | 28.4       |
| Minimum insertions in a gene             | 0           | 0          |
| Maximum insertions in a gene             | 237         | 420        |
| Genes with insertions                    | 65%         | 77%        |
| Number of genes without insertions       | 564         | 257        |
| Insertions outside of annotated genes    | 15773       | 43266      |
| Insertions in intergenic regions**       | 54.98%      | 55.66%     |



***
### <center>GeneCompare: Comparative gene analysis for the same organism</center>
***

#### Old Method

Prior to MAGenTA, comparative gene analysis between two experiments for the same organism (such as the TIGR4 in glucose and in the presence of an antibiotic) was performed by copying output files from ~the fitness script~ and calculating a t-test and average fitness difference in Excel. The resulting file would be analyzed by looking at (1) p-values that satisfy a Bonferroni adjusted critical value, (2) the number of insertions in each gene (typically >3 to be considered important) and (3) the difference between the average fitness values for the two conditions being assessed.  

#### MAGenTA method

GeneCompare automates this process by taking output files from ~the fitness script~ to perform comparative statistical tests and output a gene comparison file in less than five seconds for approximately two-thousand genes. To facilitate and extend analysis capabilities, the Bonferroni (to control family-wise error rate) and Benjamini-Hochberg (to control false discovery rate) methods were used to directly correct p-values for multiple corrections. The Bonferroni method is more conservative, increasing the probability of false negatives, while the Benjamini-Hochberg method typically determines more genes to be significant (Fig. 1). A user can decide which correction method to use based on the cost of false positives and the potential for downstream experimental validation. 

#### Usage

```perl
perl compGenes.pl <options> [aggregateFile1.csv aggregateFile2.csv OR -d directory]
```

    -i input files separted by a comma #Margaret--> EDIT THIS OPTION
    -d directory of input files
    -h prints usage and ends program
    -d directory containing input files. Make sure "/" is included after name
    -o output file for comparison data. Default: compFile1File2.csv
    -s sort output by this index of the file (indices begin at 0). Default: by mean
    -r round final output numbers to this number of decimals
    -l labels for for compared files. Used for column names and default output file name





***
### <center>StrainCompare: Comparative gene analysis for different organisms</center>
***

The StrainCompare tool, a variation of GeneCompare, is specifically for gene comparisons across two experiments from <i>different</i> organisms/strains. Since gene identification numbers will be different at the organism/strain level, a file containing the homologous genes is required. The same output as GeneCompare will be produced and can be used in downstream analysis tools. An automated method for extracting homologous genes will be a useful future improvement to this tool.

#### USAGE

```perl
perl compStrains.pl <options> -c conversion.csv [aggregateFile1.csv aggregateFile2.csv OR -d directory]
```

    -i input files separted by a comma #Margaret--> EDIT THIS OPTION
    -d directory of input files
    -c a tab delimited file with homologous genes
    -h prints usage and ends program
    -d directory containing input files. Make sure "/" is included after name
    -o output file for comparison data. Default: compFile1File2.csv
    -s sort output by this index of the file (indices begin at 0). Default: by mean
    -r round final output numbers to this number of decimals
    -l labels for for compared files. Used for column names and default output file name


***
### <center>SlidingWindow: Discovering important unannotated regions</center>
***

The method for calculating the aggregate fitness value of a gene relies on the genomic start and end coordinates of the region, which is retrievable from the organism’s Genbank record. However, not all official genome records are complete or well annotated beyond genes. At the same time, it has been shown that non-coding RNAs have an important role in virulence and other functions. Therefore, to enable discovery of such regions that may have an important fitness value to the organism, we implemented a sliding window approach to mine Tn-Seq data that falls outside of annotated genomic regions. The SlidingWindow tool first aggregates genome-wide Tn-Seq data from multiple libraries. 

#### Determining window size and step
The entire genome and ordered Tn-Seq data is scanned by performing assessments of regions at a set size (base pair length of window) and step (base pairs between the start of each window). Based on the genome used, typically a window size between 250 bp and 500 bp is small enough to pick up intergenic regions or gene domains but large enough to contain TA sites and insertions. However, this can vary depending on organism and insertion density. The DataOverview tool can help determine these factors.
</break>

#### Method
In each regional profile, the aggregate fitness cost for insertions and the insertion representation in that region is calculated. To assess insertion representation, we follow the method used by Zhang et. al.14. For a given region, with x insertions and n TA sites, 10,000 random sets of n TA sites are selected and the average of insertions over TA sites is calculated for each. The 10,000 resulting means create a null distribution. The p-value can be derived for the original region of interest by ranking it on the distribution for x/n. A statistically underrepresented number of insertions for a region will be significant if the p-value is below the critical value. Gene annotations are also recorded if they are contained in the region. In the future, a function to identify promoter regions based on -10 and -35 box sequences and domain regions will be useful. 

#### Run time
SlidingWindow performs the fitness and insertion representation assessments in approximately 8 minutes for a two-thousand base pair genome with ~70% TA site saturation. For the Tn-Seq data from Taiwan-19F Tn-seq data from growth in both conditions, 211,151 regions (300 bp) across the 2,112,148 bp genome were profiled using this approach and later used for comparative analyses. 

#### Usage

```perl
perl slidingWindow.pl -size x -f genome.fasta -d dirOfFiles/ -r genome.gbk
```

    --size  The size of the sliding window(default=500)
    --step  The window spacing (default=10)
    --csv   Name of a file to enter the .csv output for sliding windows.
    --cutoff Don't include fitness scores with average counts (c1+c2)/2 < x (default: 0)
    --essentials Calculate genome region essentiality based on transposon insertion representation
    --outdir Specify name of new directory for all output files
    --log Send all output to a log file instead of the terminal
    --indir   Directory containing all input files (results files from calc fitness script
    --usage  Print usage
    --tan Max number of TA sites in each window---used for creating null distribution library (default: 100)
    

***
### <center>WindowCompare: Comparative analysis of small, unannotated regions </center>
***

WindowCompare takes results from the SlidingWindow tool and performs a similar comparative analysis as done on genes by GeneCompare and StrainCompare. Since start and end coordinates define windows (or regions), only comparisons between datasets from the <i>same</i> organism (same genome) can be performed. 
</break>

#### USAGE

```perl
perl compWindows.pl <options> [aggregateFile1.csv aggregateFile2.csv OR -d directory]
```

    -i input files separted by a comma #Margaret--> EDIT THIS OPTION
    -d directory of input files
    -h prints usage and ends program
    -d directory containing input files. Make sure "/" is included after name
    -o output file for comparison data. Default: compFile1File2.csv
    -s sort output by this index of the file (indices begin at 0). Default: by mean
    -r round final output numbers to this number of decimals
    -l labels for for compared files. Used for column names and default output file name

#### Filtering comparison results
Beginning with the comparison file for sliding windows (output of WindowCompare) conditionally important regions are identified either manually or with the DecisionMaker tool as done with genes. After filtering out unimportant regions, the Grouper tool can group consecutive regions that are labeled as important and the CustomWindow tool can recalculate the fitness and insertion representation in these expanded regions for higher quantitative resolution. Regions are classified as intergenic if there are no gene ids recorded for that entry. A frequency distribution of all genes for the selected regions illustrates how many regions there are for each gene and how many are intergenic. 
</break>

#### Example from thesis ~insert fig~

In the example shown, over 200,000 300-bp windows for Tn-Seq data from Taiwan-19F grown in glucose and the presence of daptomycin are assessed for conditionally important regions using the adjusted p-value (FDR) and fitness difference between the two conditions (Fig.2). As shown in the yellow highlighted area, 583 regions met the criteria for having adjusted p-values below a critical value of 0.01 and a fitness difference greater than 0.1. Distributions of regions in 20 unique genes and two intergenic regions are shown in the table (Fig. 2). 
	


***
### <center>GvizTnSeq: Track visualization of Tn-Seq multi-feature data</center>
***
 	
To visualize a given region’s sequence, annotation, and insertions we use an adapted R package, Gviz, in R. The MAGenTA pipeline adaptation consists of three main functions that draw upon ones already built in the package. However, first file types must be formatted for input as track features in Gviz. For the main data track, multiple library files containing the insertion coordinates and fitness values are aggregated to produce a file containing unique insertions with a single fitness value using the SingleFit tool. For the annotation track, gene coordinates, transcription direction, and id are extracted from the Genbank record using the GetGeneFeatures tool and features specifically formatted for Gviz input.  Finally, the sequence track accepts the genome fasta file. Once these three files are uploaded in Rstudio, the customized Tn-Seq functions can be used to visualize regions by gene id or start and end coordinates with at most two experiments in the same window (Fig.3). 
</break>

#### USAGE


```r

getTracks<-function(fromCoord,toCoord,x1,x2)
```
    
    fromCoord  start genomic coordinate for the highlighted region
    toCoord    end genomic coordinate for the highlighted region
    x1         start coordinate for the viewing window (default: fromCoord)
    x2         end coordinate for the viewing window (default: toCoord)
    

#### Output

MAKE THE OUTPUT PICTURE MORE REAL

![alt text](https://github.com/antmarge/magenta/blob/master/figures/fig3A.png?raw=true "MAGenTA fig3A")

![alt text](https://github.com/antmarge/magenta/blob/master/figures/fig3B.png?raw=true "MAGenTA fig3B")






***
### <center>Using multiple MAGenTA tools for a comprehensive analysis</center>
***

Tn-Seq, coupled with MAGenTA, creates the possibility of comprehensive analysis over annotated and unannotated genomic regions by insertion representation and exact fitness assessments. Fig 3A and 3B in the GvizTnSeq section show the tetracycline resistance genes (SP_1918, SP_1919) were identified as being conditionally important for TIGR4 fitness using the GeneCompare tool. Additionally, the surrounding unannotated regions (~100 bp upstream and ~300 bp downstream) were identified as also being conditionally important for fitness in daptomycin using the SlidingWindow and WindowCompare tools. All four of these regions were easily visualized using the GvisTnSeq functions in R (Fig. 3A). These regions also have biological significance. It has been show that tetracycline resistance genes are important for pathogens, such as Staphylococcus aureus and Streptococcus pneumoniae, in daptomycin resistance. The unannotated upstream region could potentially be a promoter or enhancer that affects transcription. 
</break>

MAGenTA tools also effectively identify essential regions, which have a statistically underrepresented number of insertions, and gene domains (Fig. 3B). In Taiwan-19F,the SlidingWindow tool identified a single essential ~300 bp region within non-essential gene, (SPT_1238), endo-beta-N-acetylglucosaminidase, which is involved in processing free oligosaccharides in the cytosol.24 The presence of insertions in the gene surrounding this region and the lack of a significant aggregate fitness defect among those insertions indicates that only this domain is essential. An alignment search for the amino acid sequence of SPT_1238 in NCBI Blast revealed the presence of a glucan-binding domain (1,124 bp), which completely overlapped the essential region. The glucan-binding domain is a highly conserved region involved in carbohydrate transport. When performing a comparative strain analysis, we found that the glucan-binding domain in the TIGR4 homolog (SP_0965) is also essential, showing a highly similar insertion distribution as in SPT_1238 (Fig.3B). S. pneumoniae relies exclusively on carbohydrates as a carbon source and uses 30% of its transporters for carbohydrate uptake and several carbohydrate transporters have been shown to be necessary for S. pneumoniae survival and pathogenesis.25 The presence of this single essential domain within a non-essential gene is consistent with literature which suggests the importance of carbohydrate transport but not the endo-beta-N-acetylglucosaminidase protein itself.
</break>

Upstream of these homologs in TIGR4 and Taiwan-19F are two more sets of homologs: (1) SP_0967 and SPT_1236, which are essential genes and (2) SP_0968 and SPT_1235, of which only SP_0968 appears to be essential while SPT_1235 has a low aggregate fitness (Fig. 3B). 